# Case-3: Fine-tuning
        
        [Split planes metadata into source and target according to the location of the data (Southern/Northern)
        Taking Sourthen as a source and Northern as a target
        source.csv = source metadata
        target.csv = target metadata
        setDifferenceNorthern-Southern_lc_t=5.csv = targeted images metadata (from target dataset)]
        
        sourceCNN.h5 = pre-trained model from source (Source(s+n)_target(t-n).ipynb)
        
        -- All the layers are frozen accept second convolutional layer
        
        **metadata is used only for splitting.

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import random
import csv

In [ ]:
# Planes dataset

data = pd.read_json('planesnet.json')

In [ ]:
# Add an ID

data['ID'] = data.index

In [ ]:
data.head()

In [ ]:
# Targeted images ID for targeted transfer learning

targeted_images = pd.read_csv("setDifferenceNorthern-Southern_lc_t=5.csv")

In [ ]:
len(targeted_images)

In [ ]:
#Target metadata - collecting target planes ID

target_planes = pd.read_csv("target.csv")

In [ ]:
# Target data for targeted transfer learning [set difference]

for d in range(len(targeted_images)):
    target_planes = target_planes[target_planes.ImageID != targeted_images['ImageID'][d]]

In [ ]:
# reset the index

target_planes = target_planes.reset_index()

In [ ]:
len(target_planes)

In [ ]:
target_planes

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
target_train, target_test= train_test_split(target_planes, test_size=0.4)

In [ ]:
target_val, target_test= train_test_split(target_test, test_size=0.5)

In [ ]:
len(target_train),len(target_val),len(target_test)

In [ ]:
target_train = target_train.reset_index()
target_val = target_val.reset_index()
target_test = target_test.reset_index()

In [ ]:
train = []
for d in range(len(target_train)):
    train.append(data.iloc[target_train['ImageID'][d]])

In [ ]:
val = []
for d in range(len(target_val)):
    val.append(data.iloc[target_val['ImageID'][d]])

In [ ]:
test = []
for d in range(len(target_test)):
    test.append(data.iloc[target_test['ImageID'][d]])

In [ ]:
x_train=[]
y_train=[]
for d in range(len(train)):
    a=np.array(train[d]['data'])
    x_train.append(a.reshape((3,20*20)).T.reshape((20,20,3)))
    y_train.append(np.array(train[d]['labels']))
x_train = np.array(x_train)
y_train=np.array(y_train)

In [ ]:
x_val=[]
y_val=[]
for d in range(len(val)):
    a=np.array(val[d]['data'])
    x_val.append(a.reshape((3,20*20)).T.reshape((20,20,3)))
    y_val.append(np.array(val[d]['labels']))
x_val = np.array(x_val)
y_val=np.array(y_val)

In [ ]:
x_test=[]
y_test=[]
for d in range(len(test)):
    a=np.array(test[d]['data'])
    x_test.append(a.reshape((3,20*20)).T.reshape((20,20,3)))
    y_test.append(np.array(test[d]['labels']))
x_test = np.array(x_test)
y_test=np.array(y_test)

In [ ]:
x_train.shape, y_train.shape, x_val.shape ,y_val.shape,x_test.shape,y_test.shape

In [ ]:
# CNN model

model = Sequential()
model.add(Conv2D(32, (2, 2), padding='same',
        input_shape=x_train.shape[1:],trainable = False, name = 'layer_1'))
model.add(MaxPooling2D(pool_size=(2, 2),trainable = False, name='layer_2'))
model.add(Activation('relu',trainable = False, name = 'layer_3'))

model.add(Conv2D(64, (2, 2),padding='same',trainable = True,name = 'layer_4_Fine_tune'))
model.add(MaxPooling2D(pool_size=(2, 2),trainable = True,name='layer_5_Fine_tune'))
model.add(Activation('relu',trainable = True,name = 'layer_6_Fine_tune'))

model.add(Dropout(0.2,trainable = False,name = 'layer_7'))

model.add(Flatten(name='layer_8'))

model.add(Dense(256,trainable = False,name='layer_N9'))
model.add(Activation('relu',trainable = False,name = 'layer_N10'))

model.add(Dropout(0.4,trainable = False,name = 'layer_N11'))
model.add(Dense(128,trainable = False,name='layer_N12'))
model.add(Activation('relu',trainable = False,name = 'layer_N13'))

model.add(Dropout(0.5,trainable = False,name = 'layer_N14'))
model.add(Dense(1, activation='sigmoid',trainable = False,name='layer_N15'))


In [ ]:
# Load pre-trained model weights

from keras.models import load_model
model.load_weights('sourceCNN.h5',by_name=True)

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                rescale=1./255,
                )

test_datagen = ImageDataGenerator(
                rescale=1./255,
                )

In [ ]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=500)
validation_generator = test_datagen.flow(x_val,y_val, batch_size=500)
test_generator = test_datagen.flow(x_test,y_test,batch_size=100,shuffle=False)

In [ ]:
epochs = 50
data_augmentation = True
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = len(train_generator),
                    epochs = epochs,
                    validation_data = validation_generator,
                    validation_steps= len(validation_generator),
                    verbose=1)

In [ ]:
# Accuracy

scores = model.evaluate_generator (test_generator, verbose=0)
print('Test accuracy:', scores[1])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = model.predict_generator(test_generator)
y_pred=y_pred>0.5
y_pred=np.asarray(y_pred,dtype=bool)

In [ ]:
# Confusion matrix

print(confusion_matrix(y_test, y_pred))

In [ ]:
# Classification report

target_names = ['No Planes', 'Planes']
print(classification_report(y_test, y_pred, target_names= target_names))